In [75]:
import pandas as pd
import json

In [76]:
df_w1 = pd.read_csv("Raw/W1_AllAppsWide_2024-11-13-4.csv")
df_w1 = df_w1.query('`session.code` == "3m87qmko" | `session.code` == "wt9ndgb1"')

df_w2 = pd.read_csv("Raw/W2_all_apps_wide_2024-12-10-2.csv")
df_w2 = df_w2.query('`session.code` == "2n8orvug"')
df_w2["participant.label"][244] = "nan" # fixing the missing label

df_w3 = pd.read_csv("Raw/W3_all_apps_wide_2025-01-29.csv")
df_w3 = df_w3.query('`session.code` == "7uy8unkt"')
df_w3["participant.label"][46] = "nan" # fixing the missing label

/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/T/ipykernel_22013/219461006.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_w2["participant.label"][244] = "nan" # fixing the missing label
/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/

In [77]:
df_w1

,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,end_app.1.player.role,end_app.1.player.payoff,end_app.1.player.group_assignment,end_app.1.player.rnumber,end_app.1.player.rnumbercheck,end_app.1.player.feedback,end_app.1.player.time_firstendpage,end_app.1.player.time_endpage,end_app.1.group.id_in_subsession,end_app.1.subsession.round_number
21,1,i6v47jry,NaN,0,1,17,conjoint_app,Welcome,2024-11-12 22:13:22.255128,1,...,NaN,0.0,1,262699,NaN,NaN,-999,-999,1,1
22,2,oqjbypyi,NaN,0,17,17,end_app,End,2024-11-12 22:30:23.985745,1,...,NaN,0.0,1,970176,970176.0,NaN,2024-11-12T22:31:58.745Z,-999,1,1
23,3,zyksik0c,NaN,0,11,17,political_app,ScaloParty,2024-11-12 22:32:06.922914,1,...,NaN,0.0,2,708967,NaN,NaN,-999,-999,1,1
24,4,gqykmx8c,NaN,0,17,17,end_app,End,2024-11-13 09:22:47.022948,1,...,NaN,0.0,1,954475,954475.0,NaN,2024-11-13T09:31:36.747Z,-999,1,1
25,5,qun7aozl,NaN,0,7,17,network_app,NetworkNamedPersons,2024-11-13 09:22:58.242938,1,...,NaN,0.0,2,118960,NaN,NaN,-999,-999,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,396,4q05avx1,NaN,0,0,17,NaN,NaN,NaN,0,...,NaN,0.0,2,303602,NaN,NaN,-999,-999,1,1
626,397,enc9biai,NaN,0,0,17,NaN,NaN,NaN,0,...,NaN,0.0,1,974019,NaN,NaN,-999,-999,1,1
627,398,hokvn6z1,NaN,0,0,17,NaN,NaN,NaN,0,...,NaN,0.0,2,424820,NaN,NaN,-999,-999,1,1
628,399,b7ctm46a,NaN,0,0,17,NaN,NaN,NaN,0,...,NaN,0.0,0,367157,NaN,NaN,-999,-999,1,1


In [78]:
def prep_function_w1(df):
    # Drop rows where 'network_app.1.player.participantcode' is NaN
    df = df.dropna(subset=['network_app.1.player.participantcode'])
    # Lowercase the 'network_app.1.player.participantcode' column
    df['network_app.1.player.participantcode'] = df['network_app.1.player.participantcode'].str.lower()


    # Identify duplicates based on 'network_app.1.player.participantcode'
    duplicates = df[df.duplicated(subset='network_app.1.player.participantcode', keep=False)]
    # Filter duplicates where 'participant._index_in_pages' > 15
    filtered_duplicates = duplicates[duplicates['participant._index_in_pages'] > 15]
    # Remove the duplicates from the original dataframe (only those where 'participantcode' is duplicated)
    df = df[~df['network_app.1.player.participantcode'].isin(duplicates['network_app.1.player.participantcode'])]
    # Concatenate the filtered duplicates back into the dataframe
    df = pd.concat([df, filtered_duplicates], axis=0, ignore_index=True)


    # Lowercase person columns ('network_app.1.player.person_1' to 'network_app.1.player.person_30')
    person_columns = [f'network_app.1.player.person_{i}' for i in range(1, 31)]
    df[person_columns] = df[person_columns].apply(lambda x: x.str.lower())
    # Replace missing values with 'x' in all person columns
    df[person_columns] = df[person_columns].fillna("x")

    for _, row in df.iterrows():
        # Other node and edge processing...
        # Check if participantcode and person_1 match
        if row["network_app.1.player.participantcode"] == row["network_app.1.player.person_1"]:
            df.loc[_, "network_app.1.player.linksrechts_self"] = row["network_app.1.player.linksrechts_1"]

    df['participant.label'] = df['network_app.1.player.participantcode']

    return df


In [79]:
def prep_function_w2(df):
    df = df.dropna(subset=['participant.label'])
    
    for i in range(1, 31):
        df[f'network_app.1.player.person_{i}'] = df[f'network_app.1.player.person_{i}'].str.lower()

    df['network_app.1.player.person_3'] = df['network_app.1.player.person_3'].str.replace("ny4", "ny3", case=False, regex=False) #added because of commentary

    # Replace missing values with "x" in columns: 'network_app.1.player.person_1', 'network_app.1.player.person_2' and 28 other columns
    df = df.fillna({'network_app.1.player.person_1': "x", 'network_app.1.player.person_2': "x", 'network_app.1.player.person_3': "x", 'network_app.1.player.person_4': "x", 'network_app.1.player.person_5': "x", 'network_app.1.player.person_6': "x", 'network_app.1.player.person_7': "x", 'network_app.1.player.person_8': "x", 'network_app.1.player.person_9': "x", 'network_app.1.player.person_10': "x", 'network_app.1.player.person_11': "x", 'network_app.1.player.person_12': "x", 'network_app.1.player.person_13': "x", 'network_app.1.player.person_14': "x", 'network_app.1.player.person_15': "x", 'network_app.1.player.person_16': "x", 'network_app.1.player.person_17': "x", 'network_app.1.player.person_18': "x", 'network_app.1.player.person_19': "x", 'network_app.1.player.person_20': "x", 'network_app.1.player.person_21': "x", 'network_app.1.player.person_22': "x", 'network_app.1.player.person_23': "x", 'network_app.1.player.person_24': "x", 'network_app.1.player.person_25': "x", 'network_app.1.player.person_26': "x", 'network_app.1.player.person_27': "x", 'network_app.1.player.person_28': "x", 'network_app.1.player.person_29': "x", 'network_app.1.player.person_30': "x"})

    return df

In [80]:
def prep_function_w3(df):
    df = df.dropna(subset=['participant.label'])
    
    for i in range(1, 31):
        df[f'network_app.1.player.person_{i}'] = df[f'network_app.1.player.person_{i}'].str.lower()

    df['network_app.1.player.person_3'] = df['network_app.1.player.person_3'].str.replace("ny4", "ny3", case=False, regex=False) #added because of commentary

    # Replace missing values with "x" in columns: 'network_app.1.player.person_1', 'network_app.1.player.person_2' and 28 other columns
    df = df.fillna({'network_app.1.player.person_1': "x", 'network_app.1.player.person_2': "x", 'network_app.1.player.person_3': "x", 'network_app.1.player.person_4': "x", 'network_app.1.player.person_5': "x", 'network_app.1.player.person_6': "x", 'network_app.1.player.person_7': "x", 'network_app.1.player.person_8': "x", 'network_app.1.player.person_9': "x", 'network_app.1.player.person_10': "x", 'network_app.1.player.person_11': "x", 'network_app.1.player.person_12': "x", 'network_app.1.player.person_13': "x", 'network_app.1.player.person_14': "x", 'network_app.1.player.person_15': "x", 'network_app.1.player.person_16': "x", 'network_app.1.player.person_17': "x", 'network_app.1.player.person_18': "x", 'network_app.1.player.person_19': "x", 'network_app.1.player.person_20': "x", 'network_app.1.player.person_21': "x", 'network_app.1.player.person_22': "x", 'network_app.1.player.person_23': "x", 'network_app.1.player.person_24': "x", 'network_app.1.player.person_25': "x", 'network_app.1.player.person_26': "x", 'network_app.1.player.person_27': "x", 'network_app.1.player.person_28': "x", 'network_app.1.player.person_29': "x", 'network_app.1.player.person_30': "x"})

    return df

In [81]:
def shiny_dataframe(df):
    """
    Preprocesses the input DataFrame by performing column drops and replacing missing values.

    Args:
        df (pd.DataFrame): The input DataFrame to preprocess.

    Returns:
        pd.DataFrame: The preprocessed DataFrame.
    """
    # Columns to drop
    drop_columns = [
        'participant._is_bot', 'participant._index_in_pages', 'conjoint_app.1.player.participant_label', 'conjoint_app.1.player.id_in_group',
        'participant._current_app_name', 'participant._current_page_name', 'conjoint_app.1.player.payoff', 'demographic_app.1.player.id_in_group',
        'participant.visited', 'participant.mturk_worker_id', 'participant.mturk_assignment_id',
        'participant.payoff', 'session.label', 'session.mturk_HITId', 'session.mturk_HITGroupId',
        'session.comment', 'session.is_demo', 'session.config.real_world_currency_per_point',
        'session.config.participation_fee', 'conjoint_app.1.player.language',
        'conjoint_app.1.group.id_in_subsession', 'conjoint_app.1.subsession.round_number',
        'demographic_app.1.player.role', 'demographic_app.1.player.payoff',
        'network_app.1.player.role', 'network_app.1.player.payoff',
        'demographic_app.1.group.id_in_subsession', 'demographic_app.1.subsession.round_number',
        'conjoint_app.1.player.role', 'session.config.name',
        'network_app.1.group.id_in_subsession', 'network_app.1.subsession.round_number',
        'political_app.1.player.role', 'political_app.1.player.id_in_group',
        'political_app.1.subsession.round_number', 'political_app.1.group.id_in_subsession',
        'end_app.1.player.role', 'end_app.1.player.payoff', 'end_app.1.player.group_assignment',
        'end_app.1.player.time_endpage', 'end_app.1.group.id_in_subsession',
        'end_app.1.subsession.round_number', 'political_app.1.player.payoff'
    ]

    # Columns with missing values to replace
    fill_values_zero = {
        'demographic_app.1.player.rent': 0,
        'demographic_app.1.player.income': 0,
        'participant.label': 0,
        'political_app.1.player.sunday_poll': 0,
        'political_app.1.player.sunday_party_vote': 0,
        'political_app.1.player.sunday_not_eligible': 0,
        'political_app.1.player.noteligible_sunday_party_vote': 0,
        'demographic_app.1.player.social_networks_1': 0,
        'demographic_app.1.player.social_networks_2': 0,
        'demographic_app.1.player.social_networks_3': 0,
        'demographic_app.1.player.social_networks_4': 0,
        'demographic_app.1.player.social_networks_5': 0,
        'demographic_app.1.player.social_networks_6': 0,
        'demographic_app.1.player.social_networks_7': 0,
        'demographic_app.1.player.social_networks_8': 0,
        'demographic_app.1.player.social_networks_9': 0,
        'demographic_app.1.player.social_networks_10': 0,
        'demographic_app.1.player.participation_demonstration_1': 0,
        'demographic_app.1.player.petition_signatory_1': 0,
        'conjoint_app.1.player.participant_label': 0,
        'network_app.1.player.participantcode': 0,
        'political_app.1.player.reason_no_vote': 0
    }

    fill_values_string_zero = {
        'demographic_app.1.player.social_networks_11': "0",
        'demographic_app.1.player.study_program_other': "0"
    }

    fill_values_negative = {
        'political_app.1.player.scalo_cdu': -999,
        'political_app.1.player.scalo_csu': -999,
        'political_app.1.player.scalo_spd': -999,
        'political_app.1.player.scalo_gruene': -999,
        'political_app.1.player.scalo_fdp': -999,
        'political_app.1.player.scalo_afd': -999,
        'political_app.1.player.scalo_linke': -999,
        'political_app.1.player.scalo_bsw': -999,
        'demographic_app.1.player.edu_mother': -999,
        'demographic_app.1.player.study_program': -999
    }

    # Drop specified columns
    df = df.drop(columns=[col for col in drop_columns if col in df.columns])

    # Replace missing values with 0
    df = df.fillna({col: val for col, val in fill_values_zero.items() if col in df.columns})

    # Replace missing values with "0"
    df = df.fillna({col: val for col, val in fill_values_string_zero.items() if col in df.columns})

    # Replace missing values with -999
    df = df.fillna({col: val for col, val in fill_values_negative.items() if col in df.columns})

    # Inconsistency in no answer variable
    df = df.replace(-888, -999)

    # List of prefixes to keep
    prefixes = ['conjoint', 
                'demographic', 
                'political', 
                'vignette', 
                'participant.label', 
                'network_app.1.player.participantcode',
                'network_app.1.player.linksrechts_self',
                'end_app.1.player.catdog',
                'end_app.1.player.rnumber']

    # Filter columns by prefixes
    df = df.loc[:, df.columns.str.startswith(tuple(prefixes))]

    return df

In [82]:
df_w3

,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,end_app.1.player.payoff,end_app.1.player.group_assignment,end_app.1.player.rnumber,end_app.1.player.rnumbercheck,end_app.1.player.feedback,end_app.1.player.catdog,end_app.1.player.time_firstendpage,end_app.1.player.time_endpage,end_app.1.group.id_in_subsession,end_app.1.subsession.round_number
0,1,491x3me2,xtk,0,22,22,end_app,End,2025-01-28 14:07:06.224129,1,...,0.0,1,743237,743237.0,NaN,2.0,2025-01-28T14:25:56.707Z,-999,1,1
1,2,tran0vrr,pgx,0,22,22,end_app,End,2025-01-28 14:08:45.869348,1,...,0.0,0,589771,589771.0,NaN,2.0,2025-01-28T14:19:42.147Z,-999,1,1
2,3,th3kdh78,amu,0,22,22,end_app,End,2025-01-28 14:10:11.983747,1,...,0.0,1,759942,759942.0,NaN,2.0,2025-01-28T14:15:19.185Z,-999,1,1
3,4,c4fci2i2,qx3,0,22,22,end_app,End,2025-01-28 14:10:24.331086,1,...,0.0,2,439273,439273.0,NaN,2.0,2025-01-28T14:29:25.046Z,-999,1,1
4,5,kvxfy3uf,3eh,0,22,22,end_app,End,2025-01-28 14:11:35.438489,1,...,0.0,1,310606,310606.0,NaN,1.0,2025-01-28T14:34:28.265Z,-999,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,5dnqhh4e,NaN,0,0,22,NaN,NaN,NaN,0,...,0.0,0,444214,NaN,NaN,NaN,-999,-999,1,1
296,297,54cj5rzt,NaN,0,0,22,NaN,NaN,NaN,0,...,0.0,0,305806,NaN,NaN,NaN,-999,-999,1,1
297,298,b95nuxx5,NaN,0,0,22,NaN,NaN,NaN,0,...,0.0,1,932932,NaN,NaN,NaN,-999,-999,1,1
298,299,yxa7xrli,NaN,0,0,22,NaN,NaN,NaN,0,...,0.0,1,287331,NaN,NaN,NaN,-999,-999,1,1


In [83]:
df_w1 = prep_function_w1(df_w1)
df_w2 = prep_function_w2(df_w2)
df_w3 = prep_function_w3(df_w3)

/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/T/ipykernel_22013/32931137.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['network_app.1.player.participantcode'] = df['network_app.1.player.participantcode'].str.lower()
/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/T/ipykernel_22013/564018429.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'network_app.1.player.person_{i}'] = df[f'network_app.1.player.person_{i}'].str.lower()
/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/T/ipykernel_22013/56

In [84]:
df_filtered_w1 = shiny_dataframe(df_w1)
df_filtered_w2 = shiny_dataframe(df_w2)
df_filtered_w3 = shiny_dataframe(df_w3)

# Export dataframes as CSV
df_filtered_w1.to_csv('Cooked/df_w1_prepared.csv', index=False)
df_filtered_w2.to_csv('Cooked/df_w2_prepared.csv', index=False)
df_filtered_w3.to_csv('Cooked/df_w3_prepared.csv', index=False)

/var/folders/jl/tj67gv9n4_sd4yxc6ynl78500000gn/T/ipykernel_22013/64920226.py:80: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({col: val for col, val in fill_values_zero.items() if col in df.columns})


***
## Network Dictionaries
`dict_nodes = {participant_label : {participant_attributes : values, network_app.1.player.person_1...30 : {Friends : True, Politics : True}}}`

Note: We loose a lot of observation because of wrong entries for participantcodes of aquaintances 

In [85]:
def replace_missing_with_default(value, default_value=-999):
    """
    Replace missing values (None, NaN, or empty) with a default value.
    """
    if pd.isna(value) or value == "" or value is None:
        return default_value
    return value

def data_extraction(df, unique_id, wave):
    # Initialize nodes with empty dictionaries
    dict_nodes = {label: {} for label in df[unique_id]}
    dict_edges = {label: {} for label in df[unique_id]}
    dropped = 0
    list_dropped = []
    edge_info = {
        "friend": 0,
        "value": 0,
        "politics": 0,
        "study": 0,
        "council": 0,
        "leftright": 0,
        "sentiment": 0
    }

    # Iterate through rows to populate node and edge attributes
    for _, row in df.iterrows():
        # Add node-specific attributes
        label = row[unique_id]
        if label in dict_nodes:
            dict_nodes[label]["leftrightself"] = replace_missing_with_default(row["network_app.1.player.linksrechts_self"])
            dict_nodes[label]["gender"] = replace_missing_with_default(row["demographic_app.1.player.gender"])
            dict_nodes[label]["income"] = replace_missing_with_default(row["demographic_app.1.player.income"])
            dict_nodes[label]["rent"] = replace_missing_with_default(row["demographic_app.1.player.rent"])
            dict_nodes[label]["grade"] = replace_missing_with_default(row["demographic_app.1.player.grade"])
            dict_nodes[label]["partyvote"] = replace_missing_with_default(row['political_app.1.player.sunday_party_vote'])
            dict_nodes[label]["age"] = replace_missing_with_default(row['demographic_app.1.player.age'])
            try:
                dict_nodes[label]["ocu_father"] = replace_missing_with_default(row['demographic_app.1.player.ocu_father'])
                dict_nodes[label]["ocu_mother"] = replace_missing_with_default(row['demographic_app.1.player.ocu_mother'])
                dict_nodes[label]["edu_father"] = replace_missing_with_default(row['demographic_app.1.player.edu_father'])
                dict_nodes[label]["edz_mother"] = replace_missing_with_default(row['demographic_app.1.player.edu_mother'])    
            except KeyError:
                pass    
            # add more attributes here

        # Process connections and add edge-specific attributes
        for i in range(1, 30):
            target_person = row[f"network_app.1.player.person_{i}"]
            if target_person != "x" and target_person != label:
                if target_person in dict_nodes:
                    dict_edges[label][target_person] = {
                        "friend": replace_missing_with_default(row[f"network_app.1.player.friend_{i}"]) == 1,
                        "value": replace_missing_with_default(row[f"network_app.1.player.value_{i}"]) == 1,
                        "politics": replace_missing_with_default(row[f"network_app.1.player.politics_{i}"]) == 1,
                        "study": replace_missing_with_default(row[f"network_app.1.player.study_{i}"]) == 1,
                        "council": replace_missing_with_default(row[f"network_app.1.player.council_{i}"]) == 1,
                        "leftright": replace_missing_with_default(row[f"network_app.1.player.linksrechts_{i}"])
                    }
                    try:
                        dict_edges[label][target_person]["sentiment"] = replace_missing_with_default(row[f"network_app.1.player.sentiment_{i}"])
                    except KeyError:
                        pass

                    # Update edge information counts
                    edge_info["friend"] += dict_edges[label][target_person]["friend"]
                    edge_info["value"] += dict_edges[label][target_person]["value"]
                    edge_info["politics"] += dict_edges[label][target_person]["politics"]
                    edge_info["study"] += dict_edges[label][target_person]["study"]
                    edge_info["council"] += dict_edges[label][target_person]["council"]
                    edge_info["leftright"] += 1  # Assuming each edge has a leftright value
                    if "sentiment" in dict_edges[label][target_person]:
                        edge_info["sentiment"] += 1
                else:
                    # Log dropped connections
                    dropped += 1
                    list_dropped.append(target_person)


    # Save nodes and edges to JSON files
    with open(f'NA/nodes_{wave}.json', 'w') as json_file:
        json.dump(dict_nodes, json_file, indent=4)

    with open(f'NA/edges_{wave}.json', 'w') as json_file:
        json.dump(dict_edges, json_file, indent=4)

    # Print edge information summary
    print(f"Edge information for wave {wave}:")
    for key, value in edge_info.items():
        print(f"{key}: {value}")

    sum=0
    for key,value in dict_edges.items():
        #print(key)
        sum += len(value)
    print("Amount of Edges dropped:", len(list_dropped))

    return dict_nodes, dict_edges, list_dropped


In [86]:
dict_nodes_w1, dict_edges_w1, list_dropped_w1 = data_extraction(df_w1, 'network_app.1.player.participantcode', "W1")
dict_nodes_w2, dict_edges_w2, list_dropped_w2 = data_extraction(df_w2, "participant.label", "W2")
dict_nodes_w3, dict_edges_w3, list_dropped_w3 = data_extraction(df_w3, "participant.label", "W3")

Edge information for wave W1:
friend: 375
value: 277
politics: 242
study: 282
council: 367
leftright: 677
sentiment: 0
Amount of Edges dropped: 87
Edge information for wave W2:
friend: 436
value: 282
politics: 232
study: 227
council: 368
leftright: 720
sentiment: 720
Amount of Edges dropped: 54
Edge information for wave W3:
friend: 417
value: 296
politics: 246
study: 220
council: 372
leftright: 689
sentiment: 689
Amount of Edges dropped: 25


In [87]:
df_w2

,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,end_app.1.player.role,end_app.1.player.payoff,end_app.1.player.group_assignment,end_app.1.player.rnumber,end_app.1.player.rnumbercheck,end_app.1.player.feedback,end_app.1.player.time_firstendpage,end_app.1.player.time_endpage,end_app.1.group.id_in_subsession,end_app.1.subsession.round_number
115,1,84okfjny,ddh,0,21,21,end_app,End,2024-12-10 14:26:13.717993,1,...,NaN,0.0,1,145701,145701.0,Shikano ist richtig sweet.,2024-12-10T14:53:07.343Z,-999,1,1
116,2,3ewltlvd,evh,0,21,21,end_app,End,2024-12-10 14:26:22.940875,1,...,NaN,0.0,1,843370,843370.0,NaN,2024-12-10T14:46:29.254Z,-999,1,1
117,3,c4upbpk6,bs3,0,21,21,end_app,End,2024-12-10 14:26:23.789402,1,...,NaN,0.0,2,516998,516998.0,NaN,2024-12-10T14:49:12.511Z,-999,1,1
118,4,8oe49gr2,k4w,0,21,21,end_app,End,2024-12-10 14:26:24.290600,1,...,NaN,0.0,1,845841,845841.0,NaN,2024-12-10T14:47:16.827Z,-999,1,1
119,5,vxaikncq,39b,0,21,21,end_app,End,2024-12-10 14:26:26.451493,1,...,NaN,0.0,2,974915,974915.0,NaN,2024-12-10T14:48:57.760Z,-999,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,176,19z3qj2g,ny3,0,21,21,end_app,End,2024-12-10 14:46:40.566631,1,...,NaN,0.0,2,704460,704460.0,NaN,2024-12-10T14:58:35.558Z,-999,1,1
291,177,bv7kcmir,vkv,0,21,21,end_app,End,2024-12-10 14:46:58.312782,1,...,NaN,0.0,0,986136,986136.0,Ich habe versehentlich die falsche Nummer der ...,2024-12-10T15:00:43.217Z,-999,1,1
292,178,3cij44nl,gcu,0,21,21,end_app,End,2024-12-10 14:51:43.674573,1,...,NaN,0.0,0,527136,527136.0,NaN,2024-12-10T15:58:37.447Z,-999,1,1
293,179,kwvcl594,knc,0,21,21,end_app,End,2024-12-10 14:53:25.253053,1,...,NaN,0.0,0,166579,166579.0,NaN,2024-12-10T16:20:40.284Z,-999,1,1
